# Operational Wind Lidar Processing with LROSE

## About NSF NCAR Integrated Sounding System (ISS)

The [Integrated Sounding System](https://www.eol.ucar.edu/observing_facilities/iss) is a suite of instruments for profiling the atmosphere from ground level. Instrumentation we operate includes wind profiling radars, scanning wind lidars, sounding systems, and more. The ISS is a requestable facility that deploys for field projects around the world.

<figure align="center">
<img src="../images/lidar_tutorial_images/ISS_DG_pano_labelled.png"/>
<figcaption align = "center"> A labeled view of instruments at an ISS site on the island of Diego Garcia during the DYNAMO field campaign. </figcaption>
</figure>

### Scanning wind lidar
The ISS currently operates a [Leosphere WindCube WLS300S](https://www.vaisala.com/sites/default/files/documents/WEA-MET-WindCube-Scan-Lidar-Product-Spotlight-B212058EN-A.pdf) scanning wind lidar, primarily used to generate wind profiles in the lowest few kilometers of atmosphere. 

<figure align="center">
<img src="../images/lidar_tutorial_images/iss_cfact_windcube.png"/>
<figcaption align = "center"> Installing the ISS WindCube lidar at the CFACT campaign in Heber City, Utah. </figcaption>
</figure>

### Operational data products
<figure align="center">
<img src="../images/lidar_tutorial_images/lrose_ams_tutorial_dataflow.drawio.png"/>
<figcaption align = "center"> Diagram of ISS lidar operational dataflow. </figcaption>
</figure>

We perform basic data processing in near real time during field campaigns, so users can verify that the lidar is functioning correctly and scientists can look at initial data collected. Our automated processing pipeline consists of four steps:
1. Convert from Leosphere netCDF format to CFRadial netCDF
2. Create single-scan plot files from CFRadial netCDF
3. Calculate and plot VAD winds from CFRadial netCDF RHI scans
4. Calculate and plot consensus averages from the VAD winds

## Set up

First, install the newest version of the iss-lidar package (more information below). This is done first because the kernel will need to be restarted to load the newest version of the package.

In [ ]:
!pip install --upgrade iss-lidar

In [ ]:
# python imports
import os
import glob
import datetime

# environment variables
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['PARAMS_DIR'] =  os.environ['BASE_DIR'] + '/params/lidar'
os.environ['LIDAR_RAW_DIR'] = '/home/jovyan/share/data_files_for_notebooks/m2hats/20230820'

## Sample data

This tutorial uses one day's worth of lidar data from the [M2HATS](https://www.eol.ucar.edu/field_projects/m2hats) field campaign in summer of 2023. During this project, the WindCube lidar was used mainly to derive wind profiles, so it ran PPI scans almost continuously. It also ran N-S and E-W RHI scans, horizon sector scans, and vertical stares once hourly.

The WindCube lidar produces data in netCDF format, but it's not quite CFRadial compliant. It produces one file for each scan, and identifies the scan by type (PPI, RHI, fixed, or DBS) and by scan ID, which identifies the specific parameters (azimuth or elevation, gate spacing, etc) associated with the scan. In this data, the PPI scans with scan ID 178 were designed to be used for VAD wind calculations and have an elevation of 35º.

In [ ]:

!ls ${LIDAR_RAW_DIR}/*.nc | head


## Convert Leosphere format files to CFRadial

The first step in our dataflow is to convert files to CFRadial format. This means that all subsequent processing steps are run on CFRadial data, which avoids the need for further instrument-specific processing. 

We use RadxConvert to convert to CFRadial. RadxConvert can read and convert Leosphere format netCDFs, and in the future will also be able to read and convert data from HALO lidars.

### Set up output directory for CFRadial files

In [ ]:
os.environ['LIDAR_CFRAD_DIR'] = os.environ['BASE_DIR'] + '/data/lidar/cfradial'
!rm -rf $LIDAR_CFRAD_DIR
!mkdir -p $LIDAR_CFRAD_DIR



### Run RadxConvert

In [ ]:
# note: converting all 1400 sample files will take a few minutes
!RadxConvert -f ${LIDAR_RAW_DIR}/*.nc -outdir ${LIDAR_CFRAD_DIR} -params ${PARAMS_DIR}/RadxConvert.params 


## Generate plots for each CFRadial PPI or RHI scan

### HawkEye

Operationally we use HawkEye from LROSE for this purpose. HawkEye is generally run interactively with a user interface, but it's possible to run non-interactively by supplying a virtual frame buffer such as Xvfb. 

On ISS field servers, we start Xvfb with:

 `/usr/bin/Xvfb :1000.0 -screen 0 2000x2000x24`. 

 This can be run manually, or can be set up as a systemd service to start automatically. With Xvfb running, we set the `$DISPLAY` variable when calling HawkEye so it will use the virtual frame buffer: 
 
 `env DISPLAY=:1000.0 HawkEye -params ...`

We don't currently have a solution for running HawkEye in a jupyter notebook, because of its GUI needs. Below are two example plots that we generated using HawkEye locally on an ISS server. We generally create a plot of velocity and a plot of relative beta for each lidar scan.

<figure align="center">
<img src="../images/lidar_tutorial_images/hawkeye_velocity.png"/>
<figcaption align = "center"> An example HawkEye plot of lidar velocity. </figcaption>
</figure>

<figure align="center">
<img src="../images/lidar_tutorial_images/hawkeye_beta.png"/>
<figcaption align = "center"> An example HawkEye plot of lidar relative beta. </figcaption>
</figure>

## `iss-lidar` python package

We created our own python package to do VAD calculations and consensus averaging on wind lidar data. That code is on [github](https://github.com/ncar/iss-lidar) and newly available as a python package on [PyPi](https://pypi.org/project/iss-lidar/). This code is very much a work in progress, so please feel free to get in touch with questions, issues, comments, or pull requests.

### Scripts

This tutorial demonstrates using iss-lidar as a library, but the repo also includes two standalone scripts, one to convert PPIs to VAD winds (`ppi_to_vad.py`), and the other to convert VAD winds to consensus averaged winds (`vad_to_consensus.py`). The help output from those scripts demonstrates what command line arguments each requires.

In [ ]:
from iss_lidar import VADSet
from iss_lidar import ConsensusSet
from iss_lidar import create_cns_filename
from iss_lidar import create_filename
from iss_lidar import time_height_plot

## Generate VAD winds for each RHI file

To get a vertical profile of 3-dimensional wind speed and direction, the iss-lidar package uses the velocity-azimuth-display (VAD) algorithm. We follow the implementation as described in [Newsom et al. 2019](https://doi.org/10.2172/1238069).

The iss-lidar code reads in CFRadial PPI scans and produces a daily netCDF file containing VAD winds from all the PPI scans from that day. We use the format included in Appendix A of Newsom et al. 2019 for our VAD winds (and consensus winds) output.

### Set up output directory

In [ ]:
os.environ['LIDAR_VAD_DIR'] = os.environ['BASE_DIR'] + '/data/lidar/vad'
!rm -rf $LIDAR_VAD_DIR
!mkdir -p $LIDAR_VAD_DIR


### Select input files

In [ ]:
cfrad_dir = os.environ['LIDAR_CFRAD_DIR']
ppis_for_vad = glob.glob(cfrad_dir + '/20230820/*178_PPI*') 


### Set minimum CNR threshold
We use a value of -33 for M2HATS data processing. The minimum CNR threshold is applied to the PPI scans prior to VAD calculations.

In [ ]:
min_cnr = -33


### Create VADSet object from PPI files
This reads in PPI files from the given list of files, calculates the VAD winds for each PPI scan, and combines the VADs for each scan into a VADSet object.

In [ ]:
# note: this takes a few minutes to run on an entire day's data
vadset = VADSet.from_PPIs(ppis_for_vad, min_cnr)
print("Finished creating VADSet")


### Thresholding
Thresholding on various parameters can optionally be performed on VADSet objects prior to saving output, as a way to quality control the data. Threshold values can be set for several parameters, and can be read in from a JSON config file such as the example below:

```
{
    "correlation_min": 0.8,
    "residual_max": 2.25,
    "mean_snr_min": -28.0,
    "nbeams_used_min_fraction": 0.75
}
```
VADSet objects can currently be thresholded on four quantities: correlation coefficient below threshold value, residuals above threshold value, mean SNR below threshold value, and percentage of beams used below fractional threshold value. Users can pick which parameters to threshold on by including or excluding the corresponding items from the configuration file. If a point fails thresholding from any of the given parameters, all variables at that point will be set to missing.

In [ ]:
# load threshold file
vadset.load_thresholds(os.environ['PARAMS_DIR'] + '/vad_thresholding.json')
print(vadset.thresholds)
# apply thresholds to VADSet
vadset.apply_thresholds()

### Save VADSet object to netCDF

In [ ]:
# create filename based on begin time of file
final_file_path = create_filename(vadset.stime[0], os.environ['LIDAR_VAD_DIR'], "VAD")
vadset.to_ARM_netcdf(final_file_path)

### Plot VAD winds as time-height plot

In [ ]:
plotpath = final_file_path.replace(".nc", ".png")
time_height_plot(plotpath, vadset.u, vadset.v, vadset.stime, vadset.height)

## Generate consensus averaged winds from VAD winds

To create 30-minute averaged winds from VAD winds, the iss-lidar package uses [consensus averaging](https://glossary.ametsoc.org/wiki/Consensus_averaging). Consensus averaging is done separately on wind u, v, and w components, then the wind speed and direction are recalculated from the consensus averaged components. Median values of correlation, residual, and mean SNR are included for the subset of values used for consensus averaging for wind components.

The iss-lidar code can either read in a VAD netCDF file, or use a VADSet object in memory (i.e. read in and calculated from PPIs). It outputs daily netCDF files of 30-minute consensus averages.

### Set up output directory

In [ ]:
os.environ['LIDAR_CNS_DIR'] = os.environ['BASE_DIR'] + '/data/lidar/consensus'
!rm -rf $LIDAR_CNS_DIR
!mkdir -p $LIDAR_CNS_DIR


### Set consensus averaging parameters
We need to supply the timespan to average over, which in our case is 30 minutes. We supply the timespan as a python `timedelta` object.
Consensus averaging takes the average of all data points within a timespan that are within a specific interval of each other, so we also need to specify the interval (or window) to use. For M2HATS we used a consensus window of 5 m/s.

In [ ]:
cns_averaging_minutes = 30
cns_timespan = datetime.timedelta(minutes=cns_averaging_minutes)
cns_window = 5 

### Create ConsensusSet object from VADs
This can be done by reading in a VAD netCDF file, as shown, or can be done directly from a previously computed VADSet object.

In [ ]:
vadfile = os.environ['LIDAR_VAD_DIR'] + '/VAD_20230820.nc'
vadset = VADSet.from_file(vadfile)
cnsset = ConsensusSet.from_VADSet(vadset, cns_window, cns_timespan)


### Save ConsensusSet object to netCDF

In [ ]:
# create filename from timespan used and begin time of file
final_file_path = create_cns_filename(cns_averaging_minutes, cnsset.stime[0], os.environ['LIDAR_CNS_DIR'])
cnsset.to_ARM_netcdf(final_file_path)


### Plot consensus winds as time-height plot

In [ ]:
plotpath = final_file_path.replace(".nc", ".png")
time_height_plot(plotpath, cnsset.u, cnsset.v, cnsset.stime, cnsset.height)